In [2]:
!git clone https://github.com/Stanford-AIMI/GREEN.git

Cloning into 'GREEN'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 129 (delta 45), reused 61 (delta 18), pack-reused 0
Receiving objects: 100% (129/129), 62.47 KiB | 12.49 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [3]:
%cd GREEN
!pip install -e .

/content/GREEN
Obtaining file:///content/GREEN
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014

In [1]:
import json
eval_results_path = '/content/drive/MyDrive/lmms-finetune/data/eval_results.json'
test_results_path = '/content/drive/MyDrive/lmms-finetune/data/test_results.json'
eval_data_path = '/content/drive/MyDrive/lmms-finetune/data/annotation_quiz_all_val.json'
test_data_path = '/content/drive/MyDrive/lmms-finetune/data/annotation_quiz_all_test.json'

with open(eval_results_path, 'r') as f:
  eval_results = json.load(f)

with open(test_results_path, 'r') as f:
  test_results = json.load(f)

with open(eval_data_path, 'r') as f:
  eval_data = json.load(f)

with open(test_data_path, 'r') as f:
  test_data = json.load(f)




In [2]:
from green_score.green import compute
import json

eval_lung, eval_heart,  eval_mediastinal, eval_bone = {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}
test_lung, test_heart,  test_mediastinal, test_bone = {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}, {'ref': [], 'hyp': []}
error_dict = {'id': [], 'content':[]}
for i, ele in enumerate(eval_results):
  ref = eval(eval_data[i]['conversations'][1]['value'])
  try:
    ele = eval(ele['report'])
  except:
    error_dict['id'].append(i)
    error_dict['content'].append(ele['report'])
    continue

  for key, val in ele.items():
    if key == 'lung':
      eval_lung['ref'].append(ref[key])
      eval_lung['hyp'].append(val)
    elif key == 'heart':
      eval_heart['ref'].append(ref[key])
      eval_heart['hyp'].append(val)
    elif key == 'mediastinal':
      eval_mediastinal['ref'].append(ref[key])
      eval_mediastinal['hyp'].append(val)
    elif key == 'bone':
      eval_bone['ref'].append(ref[key])
      eval_bone['hyp'].append(val)




In [3]:
for i, ele in enumerate(test_results):
  assert test_data[i]['id'] == ele['id']
  ref = eval(test_data[i]['conversations'][1]['value'])
  error_dict = {'id': [], 'content':[]}
  try:
    ele = eval(ele['report'])
  except:
    print(i)
    error_dict['id'].append(i)
    error_dict['content'].append(ele['report'])
    continue

  for key, val in ele.items():
    if key == 'lung':
      test_lung['ref'].append(ref[key])
      test_lung['hyp'].append(val)
    elif key == 'heart':
      test_heart['ref'].append(ref[key])
      test_heart['hyp'].append(val)
    elif  key == 'mediastinal':
      test_mediastinal['ref'].append(ref[key])
      test_mediastinal['hyp'].append(val)
    elif key == 'bone':
      test_bone['ref'].append(ref[key])
      test_bone['hyp'].append(val)

24
78
94
316
403
411
477


Fix output structure issue

In [4]:
error_list = [24, 78, 94, 316, 403, 411, 477]
refined_reports = report_list = [
    {
        "heart": "The cardiac silhouette is normal in size and contours.",
        "lung": "The pulmonary vandals appear slightly hyperinflated and appear within normal limits. No effusion or pneumothorax is identified. There is no focal airspace opacity to suggest pneumothorax.",
        "mediastinal": "The mediastinal silhouette is normal in size and contours.",
        "bone": "There is no evidence for acute abnormality of the skeletal structures.",
        "others": "The heart and lungs have XXXX XXXX XXXX according to the XXXX XXXX."
    },
    {
        "heart": "Normal heart size.",
        "lung": "No pneumothorax, pleural effusions or airspace disease. Scattered calcified granulomatous fluid. No focal consolidation. Mild streaky opacities in central lateral lung with mild bronchiectasis.",
        "mediastinal": "",
        "bone": "No acute bony abnormality.",
        "others": ""
    },
    {
        "heart": "Number of cardiac contours is unremarkable.",
        "lung": "The lungs are clear, without evidence of focal consolidation, pneumothorax, or pleural effusion.",
        "mediastinal": "Number of pulmonary contours is unremarkable.",
        "bone": "Mild bony patterns are present in the thoracic spine. Stable left clavicle,",
        "others": "Mild left hemidiaphragm."
    },
    {
        "heart": "Heart size within normal limits.",
        "lung": "No acute abnormality.",
        "mediastinal": "",
        "bone": "",
        "others": ""
    },
    {
        "heart": "The heart is normal in size.",
        "lung": "The lungs are clear.",
        "mediastinal": "The mediastinum is unremarkable.",
        "bone": "",
        "others": ""
    },
    {
        "heart": "Heart at normal limits",
        "lung": "Calcified left base calcified granuloma noted",
        "mediastinal": "Cardiomediastral silhouette is normal",
        "bone": "Low back sponds",
        "others": "Right hilum is blunt"
    },
    {
        "heart": "Heart appears normal",
        "lung": "Lungs are clear and expanded",
        "mediastinal": "Mediastinum appears normal",
        "bone": "Osseous structures appear normal",
        "others": "There is mild elevation in the temporal aspect of the left breast"
    }
]
for i, idx in enumerate(error_list):
  ref = eval(test_data[idx]['conversations'][1]['value'])
  ele = refined_reports[i]
  for key, val in ele.items():
    if key == 'lung':
      test_lung['ref'].append(ref[key])
      test_lung['hyp'].append(val)
    elif key == 'heart':
      test_heart['ref'].append(ref[key])
      test_heart['hyp'].append(val)
    elif  key == 'mediastinal':
      test_mediastinal['ref'].append(ref[key])
      test_mediastinal['hyp'].append(val)
    elif  key == 'bone':
      test_bone['ref'].append(ref[key])
      test_bone['hyp'].append(val)


In [9]:
model_name = 'StanfordAIMI/GREEN-RadPhi2'

compute(model_name, eval_lung['ref'], eval_lung['hyp'], 4, output_dir=f"/content/drive/MyDrive/lmms-finetune/evaluation/eval_lung")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/296 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


100%|██████████| 74/74 [19:04<00:00, 15.47s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/eval_lung/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.501998873873874 and standard variation 0.3554765026238032 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.6283783783783784. 
  Hyperexpanded lungs 

 (b) Missing a finding present in the reference: 0.38513513513513514. 
  Absence of pneumothorax. 

 (c) Misidentification of a finding's anatomic location/position: 0.9966216216216216. 
  The candidate report misplaces the bibasilar airspace opacities in the right midlung. 

 (d) Misassessment of the severity of a finding: 0.9932432432432432. 
  Incorrectly reported absence of pleural effusions. 

 (e) Mentioning a comparison that isn't in the reference: 0.972972972972973. 
  Comparison with an earlier study. 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  4.003277528930354


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

In [6]:
compute(model_name, eval_heart['ref'], eval_heart['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/eval_heart")

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/GREEN-RadPhi2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/GREEN-RadPhi2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

100%|██████████| 74/74 [16:17<00:00, 13.21s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/eval_heart/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.6306306306306307 and standard variation 0.4198261028799449 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.9324324324324325. 
  The candidate report falsely states that the heart size is moderately enlarged. 

 (b) Missing a finding present in the reference: 0.6587837837837838. 
  The candidate report does not mention the cardiomediastinal silhouette. 

 (c) Misidentification of a finding's anatomic location/position: 1.0. 
 None 

 (d) Misassessment of the severity of a finding: 0.9054054054054054. 
  The candidate report incorrectly states that the heart size is normal, while the reference report indicates that the heart is at the upper limits of normal size. 

 (e) Mentioning a comparison that isn't in the reference: 0.9966216216216216. 
  The candidate report mentions a comparison to a previous study. 

 (f) Omitting a comparison detailing a change from a prior study: 0.993

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
compute(model_name, eval_mediastinal['ref'], eval_mediastinal['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/eval_mediastinal")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/296 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


100%|██████████| 74/74 [15:16<00:00, 12.38s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/eval_mediastinal/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.26041666666666663 and standard variation 0.42652677424946217 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.8851351351351351. 
  Mediastinal contours are within normal limits 

 (b) Missing a finding present in the reference: 0.7027027027027027. 
  The candidate report is missing. 

 (c) Misidentification of a finding's anatomic location/position: 0.9966216216216216. 
  Diaphragm contours. 

 (d) Misassessment of the severity of a finding: 1.0. 
 None 

 (e) Mentioning a comparison that isn't in the reference: 0.9932432432432432. 
  The candidate report mentions that the mediastinal contours are stable, which is not mentioned in the reference report. 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  3.148935670788224


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
compute(model_name, eval_bone['ref'], eval_bone['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/eval_bone")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/296 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


100%|██████████| 74/74 [13:39<00:00, 11.07s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/eval_bone/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.1109234234234234 and standard variation 0.2976207759350144 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.8513513513513513. 
  Degenerative changes of the spine 

 (b) Missing a finding present in the reference: 0.6824324324324325. 
  The candidate report is missing. 

 (c) Misidentification of a finding's anatomic location/position: 0.9898648648648649. 
  The candidate report mentions "bilateral thoracic spine" instead of "thoracic spine". 

 (d) Misassessment of the severity of a finding: 0.9831081081081081. 
  The candidate report describes the degenerative changes in the spine as "mild", which is not specified in the reference report. 

 (e) Mentioning a comparison that isn't in the reference: 1.0. 
 None 

 (f) Omitting a comparison detailing a change from a prior study: 1.0. None.
Seconds per example:  2.823826780995807


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
model_name = 'StanfordAIMI/GREEN-RadPhi2'
compute(model_name, test_lung['ref'], test_lung['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/test_lung")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/590 [00:00<?, ? examples/s]

...

  0%|          | 0/147 [00:00<?, ?it/s]The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
148it [37:24, 15.16s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/test_lung/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.5727865213882163 and standard variation 0.35598066662432964 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.6745762711864407. 
  Low lung volumes 

 (b) Missing a finding present in the reference: 0.4542372881355932. 
  Absence of pneumothorax 

 (c) Misidentification of a finding's anatomic location/position: 0.9983050847457627. 
  Calcified granuloma in the right lower lobe instead of the right middle lobe. 

 (d) Misassessment of the severity of a finding: 0.988135593220339. 
  Incorrectly stated no pleural effusion when reference report states no pleural effusion. 

 (e) Mentioning a comparison that isn't in the reference: 0.9728813559322034. 
  Absence of pulmonary edema. 

 (f) Omitting a comparison detailing a change from a prior study: 0.9932203389830508.  Right pleural effusion increased from previous..
Seconds per example:  4.179638429011328


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
compute(model_name, test_heart['ref'], test_heart['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/test_heart")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/590 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


148it [33:05, 13.41s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/test_heart/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.7192090395480226 and standard variation 0.4111993110888488 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.9067796610169492. 
  Mild cardiomegaly 

 (b) Missing a finding present in the reference: 0.8135593220338984. 
  The candidate report does not mention the cardiac contours. 

 (c) Misidentification of a finding's anatomic location/position: 0.9966101694915255. 
  The candidate report specifies the cardiac contours, which is a specific part of the heart, while the reference report mentions the size and contour of an unspecified object. 

 (d) Misassessment of the severity of a finding: 0.940677966101695. 
  The candidate report incorrectly states that the heart size appears within normal limits, while the reference report indicates that the heart size is at the upper limits of normal. 

 (e) Mentioning a comparison that isn't in the reference: 0.988135593220339. 
  The cand

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:
compute(model_name, test_mediastinal['ref'], test_mediastinal['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/test_mediastinal")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing data...making prompts


Map:   0%|          | 0/589 [00:00<?, ? examples/s]

Done.
==== Beginning Inference ====


148it [31:59, 12.97s/it]


==== End Inference ====
Saving generated response to prompt to  /content/drive/MyDrive/lmms-finetune/evaluation/test_mediastinal/results_.csv
Computing summary ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.3313525749858517 and standard variation 0.45100010478659075 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.8471986417657046. 
  Calcified left hilar lymph nodes 

 (b) Missing a finding present in the reference: 0.7436332767402377. 
  The candidate report is missing. 

 (c) Misidentification of a finding's anatomic location/position: 0.9898132427843803. 
  The candidate report identifies the mediastinal contours as pulmonary contours. 

 (d) Misassessment of the severity of a finding: 0.9881154499151104. 
  The candidate report incorrectly states that the mediastinum is normal in size, which is not the same as maintaining stability. 

 (e) Mentioning a comparison that isn't in the reference: 0.9898132427843803. 
  The candidate report mentions that the mediastinal contours are stable from a prior examination, which is not mentioned in the reference report. 

 (f) Omitting a co

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
model_name = 'StanfordAIMI/GREEN-RadPhi2'
compute(model_name, test_bone['ref'], test_bone['hyp'], 4, output_dir="/content/drive/MyDrive/lmms-finetune/evaluation/test_bone")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/GREEN-RadPhi2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

[Summary]: Green average 0.10928126768534238 and standard variation 0.3000670746967877 
 [Clinically Significant Errors Analyses]: <accuracy>. <representative error>

 (a) False report of a finding in the candidate: 0.8828522920203735. 
  Degenerative changes of the spine 

 (b) Missing a finding present in the reference: 0.7826825127334465. 
  The candidate report is missing. 

 (c) Misidentification of a finding's anatomic location/position: 0.9932088285229203. 
  Degenerative changes of the thoracic spine were misidentified as degenerative disc disease. 

 (d) Misassessment of the severity of a finding: 0.9915110356536503. 
  The candidate report mentions "minimal degenerative changes of the spine" instead of "mild degenerative changes of the thoracic spine". 

 (e) Mentioning a comparison that isn't in the reference: 0.99830220713073. 
  The candidate report mentions unchanged soft tissue XXXX, which is not in the reference report. 

 (f) Omitting a comparison detailing a change fr

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
